# P2P solar electricity trading simulation

In [ ]:
# Import libraries
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
import numpy as np

# Simulation function: Find intersection of two curves by interpolating and solving
def find_intersection(x, y1, y2):
    """
    Parameters:
    - x: Independent variable values
    - y1: Values of the first curve
    - y2: Values of the second curve

    Returns:
    - Intersection point of the two curves
    """

    # Create interpolating functions for the two curves
    f1 = interp1d(x, y1, kind='linear', bounds_error=False, fill_value="extrapolate")
    f2 = interp1d(x, y2, kind='linear', bounds_error=False, fill_value="extrapolate")

    # Define the equation to find the intersection point
    def equation(x0):
        return f1(x0) - f2(x0)

    try:
        # Use fsolve to find the intersection point, starting guess is 0.5
        x_intersection = fsolve(equation, x0=0.5)

        # Return the intersection point if it is within the valid range [0, 1]
        return x_intersection[0] if 0 <= x_intersection[0] <= 1 else np.nan
    except:
        return np.nan

# Define the prosumer ratios to simulate (1% to 99%)
prosumer_ratios = np.arange(0.01, 1, 0.01)

# Create a new column for the optimal prosumer ratio
municipalities_df['Optimal Prosumer Ratio'] = np.nan

# Simulation for each municipality
for index, row in municipalities_df.iterrows():
    # Check if production is 0 or missing (NaN)
    if row["Production"] == 0.0000 or np.isnan(row["Production"]):
        continue

    self_consumption_ratio = [] # The avoided electricity production exported to the grid
    self_sufficiency_ratio = [] # The avoided electricity production imported from the grid

    for ratio in prosumer_ratios:

        # Calculate production based on the prosumer ratio
        production = row["Production"] * ratio / row["Prosumer Ratio"] if row["Prosumer Ratio"] != 0.0000 else 0

        # Calculate the adjusted consumption
        # 1) Consumption to covered is set to the current solar + the current nuclear power production
        municipalities_df["Adjusted Consumption"] = (municipalities_df["elec_consumption_mwh_per_year"] * 0.425)
        # 2) Incorporating seasonality into the consumption to be covered
        municipalities_df["PDR"] = ((municipalities_df["Production"] / municipalities_df["# Producing Buildings"]) /
                                                         (municipalities_df["Adjusted Consumption"] / municipalities_df["# Buildings"]))
        municipalities_df["Adjusted Consumption"] = (1.2 *
                                                         (municipalities_df["Production"] / municipalities_df["# Producing Buildings"]) *
                                                         municipalities_df["# Buildings"] /
                                                         municipalities_df["PDR"])
        consumption = row["Adjusted Consumption"]

        # Calculate energy exported and imported
        energy_exported = max(0, production - consumption)
        energy_imported = max(0, consumption - production)

        # Calculate self-consumption ratio and self-sufficiency ratio
        self_consumption_ratio = 1 - (energy_exported / production) if production != 0.0000 else 0
        self_sufficiency_ratio = 1 - (energy_imported / consumption) if consumption != 0.0000 else 0

        # Append ratios to lists
        self_consumption_ratios.append(self_consumption_ratio)
        self_sufficiency_ratios.append(self_sufficiency_ratio)

    # Find intersection of the two curves
    optimal_prosumer_ratio = find_intersection(prosumer_ratios, self_consumption_ratios, self_sufficiency_ratios)

    # Set the calculated optimal ratio to the corresponding row in municipalities_df
    municipalities_df.at[index, 'Optimal Prosumer Ratio'] = optimal_prosumer_ratio

# Save resulting data
municipalities_df.to_csv('municipalities_df.csv', index=False)